# Basics of LLMs with Ollama + Mistral


In [2]:
!pip install ipykernel
python -m ipykernel install --user --name=ollama-env --display-name "Python (ollama-env)"

SyntaxError: invalid syntax (2671665110.py, line 2)

In [3]:
import sys
sys.executable

'C:\\Users\\User\\.conda\\envs\\training_env\\python.exe'

## 2. Hello LLM — Single Prompt (Generate API)

In [5]:
import ollama

response = ollama.generate(
    model='mistral',
    prompt="Hello! What can you do?"
)
print(response['response'])

# OpenAI Equivalent
# from openai import OpenAI
# client = OpenAI()
# client.responses.create(model="gpt-4.1-mini", input="Hello! What can you do?")

# Hugging Face Equivalent (commented)
# from transformers import pipeline
# nlp = pipeline("text-generation", model="mistral")
# nlp("Hello! What can you do?")

 Hello! I'm a model of an AI assistant designed to help answer questions, provide information, assist in tasks, and engage in conversation. I can help with things like setting reminders, providing weather updates, answering trivia questions, helping with math problems, and much more. I can also help you manage your schedule, book appointments, and even control smart home devices if they are connected to the internet. Let me know how I can assist you today!


## 3. Conversation — Chat API (with Memory)

In [3]:
import ollama
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Tell me something about Mysuru."}
]

response = ollama.chat(
    model='mistral',
    messages=messages
)
print(response['message']['content'])

# Memory is retained by sending back the full messages list!
messages.append(response['message'])

# Follow-up query
messages.append({"role": "user", "content": "Where is that located?"})
response = ollama.chat(model='mistral', messages=messages)
print(response['message']['content'])

 Mysuru, also known as Mysore, is a historic city located in the southwestern state of Karnataka, India. It's renowned for its rich history, vibrant culture, and beautiful architecture. The city is particularly famous for Mysore Palace, an imposing edifice with a striking combination of Hindu, Saracenic, and Rajput architectural styles.

Mysuru also hosts the Mysore Dussehra festival, one of India's grandest and most famous cultural events celebrated every year. The city is known for its silk industry, producing some of the finest quality silk sarees in the world.

Another notable attraction is the Chamundi Hills, where the Chamundeshwari Temple dedicated to the goddess Chamundeshwari, the patron deity of Mysuru, stands atop. The city is also home to several educational institutions of national importance such as the University of Mysore and the Indian Institute of Science Education and Research (IISER).

Mysuru's pleasant climate, green surroundings, and rich cultural heritage make it

In [6]:
import ollama

response = ollama.generate(
    model="mistral",
    prompt="Explain what embeddings are in 1 sentence."
)

print(response["response"])


 Embeddings are a way of representing data points from various domains, such as text or images, as vectors in a lower-dimensional space while preserving their semantic meaning.


In [4]:
import ollama
# Try multi-turn chat
print("User: Who was the last king of Mysuru?")
res1 = ollama.generate(model="mistral", prompt="Who was the last king of Mysuru?")
print("Model:", res1["response"])

print("\nUser: Where was he born?")
res2 = ollama.generate(model="mistral", prompt="Where was he born?")
print("Model:", res2["response"])

User: Who was the last king of Mysuru?
Model:  The last Maharaja (King) of Mysore, as it is officially known today, was Sri Jayachamarajendra Wodeyar. He reigned from 1947 to 1949. However, it's important to note that the princely state of Mysore was integrated into the Indian Union in 1947 and Jayachamarajendra Wodeyar served as the Rajpramukh (Governor) of Mysore State from 1947 until 1952, when India became a republic and the position of Rajpramukh was abolished. He abdicated the throne in 1949.

User: Where was he born?
Model:  To provide an accurate answer, I would need a specific name or identity. If you're referring to a historical figure, public personality, or fictional character, please specify who you mean, and I'll be happy to help!


In [ ]:
from ollama import Client

client = Client()  # auto host = http://localhost:11434

messages = []

def chat(user_message):
    messages.append({"role": "user", "content": user_message})
    response = client.chat(model="mistral", messages=messages)
    messages.append(response["message"])
    print("Assistant:", response["message"]["content"])

chat("Who was the last king of Mysuru?")
chat("Where was he born?")


## 4. Tokens Demo

In [7]:
prompt = "Who was the last king of Mysuru?"
response = ollama.generate(model="mistral", 
                           prompt=f"Tokenize this: {prompt}")
print(response['response'])


# Tokens = cost + context length!

 Here is how the sentence "Who was the last king of Mysuru?" can be tokenized:

1. Who
2. was
3. the
4. last
5. king
6. of
7. Mysuru?

Each word or phrase (except for contractions) is a token in this context.


## 5. Embeddings (Meaning Understanding)

In [ ]:
#!pip install numpy

In [ ]:
import numpy as np

def embed(text):
    return ollama.embeddings(model="mistral", prompt=text)['embedding']

v1 = embed("king")
v2 = embed("queen")

similarity = np.dot(v1, v2)/(np.linalg.norm(v1)*np.linalg.norm(v2))
print("Similarity:", similarity)

# HF Equivalent (commented)
# from sentence_transformers import SentenceTransformer, util
# model = SentenceTransformer('all-MiniLM-L6-v2')
# sim = util.cos_sim(model.encode("king"), model.encode("queen"))
# print(sim)
print(v1)
print(v2)

## 6. Temperature (Creativity Control)

In [12]:
import ollama
q = "Write a creative line about Mysuru."
for t in [0.0, 0.7, 1.2]:
    r = ollama.chat(model="mistral",
                    messages=[{"role":"user","content":q}],
                    options={'temperature':t})
    print(f"Temp {t}: {r['message']['content']}")

Temp 0.0:  In the heart of Karnataka, where royalty whispers in every breeze, lies Mysuru, a city that dances between tradition and modernity with graceful elegance.
Temp 0.7:  In Mysuru, where the sun paints silk-city mornings with gold hues and jasmine whispers dance in the air, every moment echoes a symphony of timeless charm.
Temp 1.2:  "Mysore, where the whispering silks of Mysore Palace twirl in rhythm with the lilting melodies of a traditional Yakshagana performance, a vibrant dance of history and culture."

The city of Mysuru, renowned for its opulent palaces and rich cultural heritage, is known for many things. The famous Mysore Palace is one such landmark that reflects the grandeur and elegance of the region's history. At the same time, Yakshagana, a traditional form of theater unique to Karnataka, showcases the vibrant and vivacious spirit of the city and its people. This creative line aims to encapsulate both the historical beauty and the lively cultural essence that Mysore

## 7. Streaming Response

In [ ]:
stream = ollama.chat(
    model="mistral",
    messages=[{"role": "user", "content": "Tell me a short story about a tiger."}],
    stream=True,
)
for chunk in stream:
    print(chunk['message']['content'], end="", flush=True)

# OpenAI Equivalent
# client.responses.create(..., stream=True)

In [1]:
from ollama import Client

client = Client()

stream = client.generate(
    model="mistral",
    prompt="Explain transformers in 3 short bullets.",
    stream=True
)

for chunk in stream:
    print(chunk["response"], end="", flush=True)


1. Transformers are a type of machine learning model used for natural language processing tasks, such as translating from one language to another or generating text. They were introduced by Google researchers in 2017 and have since become one of the most popular models for NLP tasks due to their ability to capture long-range dependencies in text.

2. Transformers consist of a series of self-attention layers, which allow each word in the input sequence to attend to every other word in the sequence, weighted by the relevance of each word for understanding the current word's meaning. This allows the model to focus on relevant information and ignore irrelevant details, resulting in more accurate predictions.

3. Unlike recurrent neural networks (RNNs) and convolutional neural networks (CNNs), transformers do not use recurrence or convolution, making them faster and easier to train on long sequences. Transformers also handle parallelization better than RNNs and CNNs, allowing for more effic

## 8. Context Window Limits

In [ ]:
text = "Mysuru " * 1500
response = ollama.chat(
    model="mistral",
    messages=[{"role":"user", "content":text + "\nWhat word did I repeat?"}]
)
print(response['message']['content'])

## How Chat Memory Works
We manually maintain full conversation in `messages` list

➡ Ollama does NOT store memory internally  
➡ We send all history every time (just like OpenAI)

This leads to **context window limits** — motivation for RAG + LangChain Memory.


---
# Next: LangChain
Before jumping, remember:
| Feature | Raw Ollama | With LangChain |
|--------|------------|----------------|
| Memory | Manual messages[] | Built‑in memory modules |
| Tools & Agents | Hard to implement | 1‑line setup |
| RAG | Manual vector DB | Automated pipelines |
| Workflows | Custom code | Chains & Executors |